In [45]:
import pandas as pd
import numpy as np

import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import string

import arabicstopwords.arabicstopwords as stp

import re

import nltk
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('new.csv')
df


,links,court,court_id,court_date,appeal_court,appeal_id,appeal_date,region,city,judgment_text,appeal_text
0,القضية رقم ٤٦٤٦ لعام ١٤٤٢ هـ,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٤٦٤٦,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٨٥,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...
1,القضية رقم ٦٩٥٨ لعام ١٤٤٢ هـ,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٦٩٥٨,تاريخها: ٢٥/١٢/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٥٤٦,تاريخه: ٦/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...
2,القضية رقم ٦٦٠٧ لعام ١٤٤٢ هـ,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٦٦٠٧,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٩٠,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...
3,القضية رقم ٤٣٣١ لعام ١٤٤٢ هـ,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٤٣٣١,تاريخها: ١٨/١١/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٣٧,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...
4,القضية رقم ٩٣٩٩ لعام ١٤٤١ هـ,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٩٣٩٩,تاريخها: ١٥/١٠/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٥٣٤٣,تاريخه: ٦/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...
...,...,...,...,...,...,...,...,...,...,...,...
208,القضية رقم ٤٧٠٧ لعام ١٤٤٢ هـ,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٤٧٠٧,تاريخها: ١٨/١٢/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٣٣٥,تاريخه: ٨/٢/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...
209,قرار الدائرة الخامسة بالمحكمة العليا رقم ٤٣٨٥٠...,المحكمة: المحكمة العليا - الهيئة الدائمة,رقم القضية - القرار: ٤٣٨٥٠,تاريخها: ١٢/٢/١٤٤٣,NaN,NaN,NaN,NaN,المدينه: الرياض,الحمد لله وحده والصلاة والسلام على نبينا محمد ...,NaN
210,القضية رقم ١١١٠ لعام ١٤٤٢ هـ,المحكمة: المحكمة التجارية,رقم القضية - القرار: ١١١٠,تاريخها: ١٤/٩/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٥١٥٩,تاريخه: ٨/٢/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...
211,القضية رقم ١٥٨٢ لعام ١٤٤١ ه,المحكمة: المحكمة التجارية,رقم القضية - القرار: ١٥٨٢,تاريخها: ٢/١٢/١٤٤٢,محكمة الاستئناف: محكمة الاستئناف,رقم القرار: ١٣٦,تاريخه: ٧/٢/١٤٤٣,المدينة: المنطقة الشرقية,المدينه: الدمام,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   links          213 non-null    object
 1   court          213 non-null    object
 2   court_id       213 non-null    object
 3   court_date     213 non-null    object
 4   appeal_court   169 non-null    object
 5   appeal_id      169 non-null    object
 6   appeal_date    169 non-null    object
 7   region         169 non-null    object
 8   city           213 non-null    object
 9   judgment_text  213 non-null    object
 10  appeal_text    169 non-null    object
dtypes: object(11)
memory usage: 18.4+ KB


In [4]:
df.duplicated().sum()

0

In [5]:
df_Train, df_test = train_test_split(df, test_size=0.2, random_state= 30)

df_train, df_val = train_test_split(df_Train, test_size=0.2, random_state= 30)

In [6]:
print(f'Shape of train:', df_train.shape)
print(f'Shape of validation:', df_val.shape)
print(f'Shape of test:', df_test.shape)

Shape of train: (136, 11)
Shape of validation: (34, 11)
Shape of test: (43, 11)


In [7]:
df_train.isnull().sum()

links             0
court             0
court_id          0
court_date        0
appeal_court     28
appeal_id        28
appeal_date      28
region           28
city              0
judgment_text     0
appeal_text      28
dtype: int64

In [8]:
df_val.isnull().sum()

links            0
court            0
court_id         0
court_date       0
appeal_court     5
appeal_id        5
appeal_date      5
region           5
city             0
judgment_text    0
appeal_text      5
dtype: int64

In [9]:
df_train.appeal_court.value_counts(),

(محكمة الاستئناف: المحكمة التجارية    101
 محكمة الاستئناف: محكمة الاستئناف       7
 Name: appeal_court, dtype: int64,)

In [10]:
df_val.appeal_court.value_counts(),

(محكمة الاستئناف: المحكمة التجارية    28
 محكمة الاستئناف: محكمة الاستئناف      1
 Name: appeal_court, dtype: int64,)

In [11]:
df_train['appeal_court'].fillna(df_train['appeal_court'].mode()[0], inplace = True)

In [12]:
df_val['appeal_court'].fillna(df_val['appeal_court'].mode()[0], inplace = True)

In [13]:
df_train.appeal_id.value_counts()

رقم القرار: ١٢٢     2
رقم القرار: ٥٩١     1
رقم القرار: ٦٢٣     1
رقم القرار: ١٥٢     1
رقم القرار: ٢٤٣٠    1
                   ..
رقم القرار: ٥١٥٩    1
رقم القرار: ٣٤٧     1
رقم القرار: ٥٣٠٠    1
رقم القرار: ٢٣١٨    1
رقم القرار: ٥٢٠٥    1
Name: appeal_id, Length: 107, dtype: int64

In [14]:
df_val.appeal_id.value_counts()

رقم القرار: ٨٤      1
رقم القرار: ٣٣٧٨    1
رقم القرار: ٥١٧     1
رقم القرار: ٥٣٤٣    1
رقم القرار: ١٢٦     1
رقم القرار: ٤٢٦     1
رقم القرار: ٣٢٨١    1
رقم القرار: ٣١٠٢    1
رقم القرار: ٥٥٨١    1
رقم القرار: ٣٠٣٨    1
رقم القرار: ٣٦      1
رقم القرار: ٧٩٥     1
رقم القرار: ٤٧١٦    1
رقم القرار: ٥٤٦٧    1
رقم القرار: ٥٣٨١    1
رقم القرار: ٢٥١٤    1
رقم القرار: ٤٤٢١    1
رقم القرار: ٣٣٥     1
رقم القرار: ٧٠٩     1
رقم القرار: ٥٥٧٠    1
رقم القرار: ١٩٩٨    1
رقم القرار: ٥٤٩١    1
رقم القرار: ١٩٩٧    1
رقم القرار: ٣٩      1
رقم القرار: ٥٤٠٣    1
رقم القرار: ٥٥٢٥    1
رقم القرار: ٥٠٥٠    1
رقم القرار: ٥١٣٣    1
رقم القرار: ٢٧٠٠    1
Name: appeal_id, dtype: int64

In [15]:
df_train.dropna(subset=['appeal_id'], inplace=True);

In [16]:
df_val.dropna(subset=['appeal_id'], inplace=True);

In [17]:
df_train['appeal_date'] = df_train['appeal_date'].str.replace('تاريخه:','')

In [18]:
df_val['appeal_date'] = df_val['appeal_date'].str.replace('تاريخه:','')

In [19]:
df_train['appeal_id'] = df_train['appeal_id'].str.replace('رقم القرار:','')

In [20]:
df_val['appeal_id'] = df_val['appeal_id'].str.replace('رقم القرار:','')

In [21]:
df_val['appeal_date'] = df_val['appeal_id'].str.replace('رقم القرار:','')

In [22]:
df_train['court'] = df_train['court'].str.replace('المحكمة:','')

In [23]:
df_train['court_id'] = df_train['court_id'].str.replace('رقم القضية - القرار:','')

In [24]:
df_val['court_id'] = df_val['court_id'].str.replace('رقم القضية - القرار:','')

In [25]:
df_train['court_date'] = df_train['court_date'].str.replace('تاريخها: ','')

In [26]:
df_val['court_date'] = df_val['court_date'].str.replace('تاريخها: ','')

In [27]:
df_train['appeal_court'] = df_train['appeal_court'].str.replace('محكمة الاستئناف: ','')

In [28]:
df_val['appeal_court'] = df_val['appeal_court'].str.replace('محكمة الاستئناف: ','')

In [29]:
df_train['region'] = df_train['region'].str.replace('المدينة: ','')

In [30]:
df_val['region'] = df_val['region'].str.replace('المدينة: ','')

In [31]:
df_train['city'] = df_train['city'].str.replace('المدينه: ','')

In [32]:
df_val['city'] = df_val['city'].str.replace('المدينه: ','')

In [33]:
df_train.city.value_counts()

الرياض             88
المدينة المنورة     9
الدمام              7
بريدة               4
Name: city, dtype: int64

In [34]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
df_train.judgment_text = df_train.judgment_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
df_train.appeal_text = df_train.appeal_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))

In [35]:
df_val.judgment_text = df_val.judgment_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
df_val.appeal_text = df_val.appeal_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))

In [36]:
df_train

,links,court,court_id,court_date,appeal_court,appeal_id,appeal_date,region,city,judgment_text,appeal_text
6,القضية رقم ١٠٢٤٢ لعام ١٤٤١ هـ,المحكمة التجارية,١٠٢٤٢,٢٢/٣/١٤٤٢,المحكمة التجارية,٢١٢٨,٧/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...
11,القضية رقم ٣٨٩٥ لعام ١٤٤٢ هـ,المحكمة التجارية,٣٨٩٥,٢٣/٩/١٤٤٢,المحكمة التجارية,٢٤٥,٥/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...
1,القضية رقم ٦٩٥٨ لعام ١٤٤٢ هـ,المحكمة التجارية,٦٩٥٨,٢٥/١٢/١٤٤٢,المحكمة التجارية,٥٤٦,٦/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...
93,القضية رقم ١٠٠٧٥ لعام ١٤٤١ هـ,المحكمة التجارية,١٠٠٧٥,١٦/٣/١٤٤٢,المحكمة التجارية,٢٥٠٠,٧/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...
56,القضية رقم ٧٤٤٠ لعام ١٤٣٧ هـ,المحكمة التجارية,٧٤٤٠,٢٤/٢/١٤٤٢,المحكمة التجارية,٢٢٢٩,٦/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...
...,...,...,...,...,...,...,...,...,...,...,...
163,القضية رقم ١٣٠٨ لعام ١٤٤٢ هـ,المحكمة التجارية,١٣٠٨,١٢/١١/١٤٤٢,محكمة الاستئناف,١٢٢,٧/٢/١٤٤٣,المنطقة الشرقية,الدمام,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...
64,القضية رقم ٢٢٤٤ لعام ١٤٤٢ هـ,المحكمة التجارية,٢٢٤٤,١٢/٦/١٤٤٢,المحكمة التجارية,٣٦٩٤,٩/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...
122,القضية رقم ٨٥٣٢ لعام ١٤٣٩ هـ,المحكمة التجارية,٨٥٣٢,٣٠/٨/١٤٤٢,المحكمة التجارية,٤٧٦٧,٨/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...
140,القضية رقم ١٤٧٨ لعام ١٤٤٢ هـ,المحكمة التجارية,١٤٧٨,١٠/٤/١٤٤٢,المحكمة التجارية,٢٠٠٧,٨/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...


# Preprocessing

### Remove stopwords

In [37]:
# df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stp.stopwords_list()]))
# df_train['appeal_text'] = df_train['appeal_text'].apply(lambda y: ' '.join([word for word in y.split() if word not in stp.stopwords_list()]))


### Remove digits

In [38]:
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: re.sub('\w*\d\w*', '', x))
df_train['appeal_text'] = df_train['appeal_text'].apply(lambda x: re.sub('\w*\d\w*', '', x))

In [43]:
df_train['judgment_text'] = df_train['judgment_text'].str.replace('\n','')

In [54]:
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: ISRIStemmer().suf32(x))


In [55]:
df_train['judgment_text'][0]

'الحمد لله والصلاة والسلام على رسول ﷲ أما بعدفلدى الدائرة التجارية الثانية والعشرون وبناءً على القضية رقم   لعام  هالمقامة من عبدﷲ ابراهيم علي المحمود سجل تجاري  ضد تغريد بنت محمد  صفير هوية وطنية    القاضي عمر بن مبارك الدوسري رئيسا الوقائع تتلخص وقائع هذه الدعوى وبالقدر اللازم لإصدار هذا الحكم في أنه وردت إلى المحكمة التجارية لائحة دعوى مقدمة من المدعي وكالة ضد المدعى عليها والمثبتة بياناتها في ملف القضية حيث جاء في لائحة الدعوى اتفق موكلي مع الجهة المدعى عليها بتأريخ   لتنفيذ أعمال الموقع الخارجي لمشروع القوات البحرية الملكية السعودية في الجبيل وتمثل  من أعمال المشروع والمتمثلة في أعمال السفلتة وغرف التفتيش وعددها  غرفة ومدة العقد  خمسة عشر يوماً ولم تنجز الجهة المدعى عليها سوى  من الأعمال وعدد  غرف تفتيش من عدد  غرفة واستلموا كامل المبلغ وقدره  مئتان واثنان وعشرون الف ريال دون إتمام المشروع والمستحق الفعلي لهم هو  مئة وعشرة ألف ريالاً فقط لذا أطلب  أن يلزم الجهة المدعى عليها مؤسسة قمة الصحراء للمقاولات العامة بدفع لموكلتي مبلغ وقدره  مئة واثنا عشر ألف ريالاً فقط وبعد أن قيدت الأورا

# TF-IDF

In [56]:
cv_tfidf = TfidfVectorizer()
x = cv_tfidf.fit_transform(df_train['judgment_text']).toarray()

In [57]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [58]:
dt_tfidf = pd.DataFrame(x,columns=cv_tfidf.get_feature_names())
dt_tfidf

,aed,ikk,ilg,ip,kabbani,microsoft,ncb,ritco,rtpcr,teams,...,يوليو,يوم,يوما,يومرئيس,يومي,يوميا,يومين,يونيو,يوهمني,يي
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.022034,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.02085,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
